<a href="https://colab.research.google.com/github/cagBRT/timeSeries/blob/main/7_FBProphet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Using Facebook Prophet**

Released by Facebook in 2017, forecasting tool [Prophet](https://research.fb.com/prophet-forecasting-at-scale/) is designed for analyzing time-series that display patterns on different time scales such as yearly, weekly and daily. It also has advanced capabilities for modeling the effects of holidays on a time-series and implementing custom changepoints. 

Prophet assumes a decomposable model of 3 parts:<br>
timer series = trend + seasonality + holidays + error<br>
<br>
error is any unusual changes not accommodated by the model

In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/cagBRT/timeSeries.git cloned-repo
%cd cloned-repo

In [ ]:
!pip install --upgrade xlrd

**Import the libraries**<br>
Notice we import the Facebook open source library, Prophet. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet

**Import the dataset of furniture and office supply sales**

In [ ]:
df = pd.read_excel("Sample - Superstore.xls")
df.head()

In [ ]:
furniture = df.loc[df['Category'] == 'Furniture']
furniture.head()

Prepare two datasets, one for furniture and one for office supplies. <br>

In [ ]:
furniture = df.loc[df['Category'] == 'Furniture']
office = df.loc[df['Category'] == 'Office Supplies']
furniture.shape, office.shape

**Drop all the columns for each dataset except: Date and Sales**

In [ ]:
cols = ['Row ID', 'Order ID', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Quantity', 'Discount', 'Profit']
furniture.drop(cols, axis=1, inplace=True)
office.drop(cols, axis=1, inplace=True)

**For both datasets**, 
- group (using resample) into monthly periods of sales. <br>
- the set the index to order date

In [ ]:
furniture = furniture.sort_values('Order Date')
office = office.sort_values('Order Date')
furniture = furniture.groupby('Order Date')['Sales'].sum().reset_index()
office = office.groupby('Order Date')['Sales'].sum().reset_index()
furniture = furniture.set_index('Order Date')
office = office.set_index('Order Date')
y_furniture = furniture['Sales'].resample('MS').mean()
y_office = office['Sales'].resample('MS').mean()

**Merge the two datasets into one dataset called store**

In [ ]:
furniture = pd.DataFrame({'Order Date':y_furniture.index, 'Sales':y_furniture.values})
office = pd.DataFrame({'Order Date': y_office.index, 'Sales': y_office.values})
store = furniture.merge(office, how='inner', on='Order Date')
store.rename(columns={'Sales_x': 'furniture_sales', 'Sales_y': 'office_sales'}, inplace=True)
store.head()

**Plot the monthly furniture and office supply sales **

In [ ]:
plt.figure(figsize=(20, 8))
plt.plot(store['Order Date'], store['furniture_sales'], 'b-', label = 'furniture')
plt.plot(store['Order Date'], store['office_sales'], 'r-', label = 'office supplies')
plt.xlabel('Date'); plt.ylabel('Sales'); plt.title('Sales of Furniture and Office Supplies')
plt.legend();

**Rename the columns in the datasets**

In [ ]:
furniture = furniture.rename(columns={'Order Date': 'ds', 'Sales': 'y'})
office = office.rename(columns={'Order Date': 'ds', 'Sales': 'y'})

**Call the model Prophet and train it on the datasets**

In [ ]:
furniture_model = Prophet(interval_width=0.95)
furniture_model.fit(furniture)

office_model = Prophet(interval_width=0.95)
office_model.fit(office)

**Use the model to make predictions about future furniture and office supply sales**

In [ ]:
furniture_forecast = furniture_model.make_future_dataframe(periods=36, freq='MS')
furniture_forecast = furniture_model.predict(furniture_forecast)
office_forecast = office_model.make_future_dataframe(periods=36, freq='MS')
office_forecast = office_model.predict(office_forecast)

**Plot the furniture forecast**

In [ ]:
plt.figure(figsize=(18, 6))
furniture_model.plot(furniture_forecast, xlabel = 'Date', ylabel = 'Sales')
plt.title('Furniture Sales');

**Plot the office supply forecast**

In [ ]:
plt.figure(figsize=(18, 6))
office_model.plot(office_forecast, xlabel = 'Date', ylabel = 'Sales')
plt.title('Office Supplies Sales');

**Merge the two forecast datasets into one dataset called merge_furniture_forecast**

In [ ]:
furniture_forecast.head()

In [ ]:
furniture_names = ['furniture_%s' % column for column in 
                   furniture_forecast.columns]
office_names = ['office_%s' % column for column in office_forecast.columns]
merge_furniture_forecast = furniture_forecast.copy()
merge_office_forecast = office_forecast.copy()
merge_furniture_forecast.columns = furniture_names
merge_office_forecast.columns = office_names

In [ ]:
forecast = pd.merge(merge_furniture_forecast, merge_office_forecast, 
                    how = 'inner', left_on = 'furniture_ds', 
                    right_on = 'office_ds')
forecast = forecast.rename(columns=
                           {'furniture_ds': 'Date'}).drop('office_ds',axis=1)
forecast.head()

**Plot the forecasts for furniture and office sales**

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(forecast['Date'], forecast['furniture_trend'], 'b-', label="Furniture")
plt.plot(forecast['Date'], forecast['office_trend'], 'r-', label="Office Supplies")
plt.legend(); plt.xlabel('Date'); plt.ylabel('Sales')
plt.title('Furniture vs. Office Supplies Sales Trend');
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(forecast['Date'], forecast['furniture_yhat'], 'b-', label="Furniture")
plt.plot(forecast['Date'], forecast['office_yhat'], 'r-', label='Office Supplies')
plt.legend(); plt.xlabel('Date'); plt.ylabel('Sales')
plt.title('Furniture vs. Office Supplies Estimate');
plt.show()

**Plot the trend lines for furniture and office supplies**

In [ ]:
furniture_model.plot_components(furniture_forecast);

In [ ]:
office_model.plot_components(office_forecast);